In [ ]:
from data_fetch import fetch_stock_data
from strategy import apply_strategy
from backtest import backtest
from google_sheet import connect_google_sheet, log_trades
from ml_model import train_model
from telegram_bot import send_telegram
import os

os.environ["GOOGLE_SHEETS_CREDENTIALS"] = "/content/credentials.json.json"
os.environ["TELEGRAM_TOKEN"] = "8143044630:AAEKt3jjfUnuiCIqh654CBjm9md8m7nv-So"
os.environ["TELEGRAM_CHAT_ID"] = "5417266671"
stock_list = ['RELIANCE.NS', 'TCS.NS', 'INFY.NS']
data = fetch_stock_data(stock_list, '2024-01-01', '2025-02-01')
sheet = connect_google_sheet("Algo_Trading_Log")


for stock, df in data.items():
    df = apply_strategy(df)

    if df.empty or 'Buy_Signal' not in df.columns:
        print(f"{stock} - No data or signal to process.")
        continue

    if not df['Buy_Signal'].any():
        print(f"{stock} - No buy signals in strategy.")
        continue

    try:
        trades = backtest(df)
    except ValueError as ve:
        print(f"{stock} - Backtest error: {ve}")
        continue

    # Convert pandas Series to native types for Google Sheets logging
    clean_trades = []
    for trade in trades:
        clean_trade = {k: (v.item() if hasattr(v, 'item') else v) for k, v in trade.items()}
        clean_trades.append(clean_trade)

    log_trades(sheet, clean_trades)
    send_telegram(f"{stock}: {len(trades)} trades logged.")

    if len(df.dropna()) > 60:
        model, acc = train_model(df)
        print(f"{stock} - ML Accuracy: {acc:.2f}")
    else:
        print(f"{stock} - Skipping ML model: not enough data")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


✅ Telegram alert sent!
RELIANCE.NS - ML Accuracy: 0.55
✅ Telegram alert sent!
TCS.NS - ML Accuracy: 0.45
✅ Telegram alert sent!
INFY.NS - ML Accuracy: 0.64
